# 第 7 章：使用示例（少样本提示）

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格以加载您的 API 密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程

**向 Claude 提供您希望它如何表现（或不希望它如何表现）的示例，对于以下两点极其有效**：
- 获得正确答案
- 以正确的格式获得答案

这种提示方式也称为“**少样本提示**”（few-shot prompting）。您可能还会遇到“零样本”（zero-shot）、“n 样本”（n-shot）或“单样本”（one-shot）等说法。“样本”（shots）的数量指的是在提示中使用的示例数量。

### 示例

假设你是一名开发者，正在尝试构建一个“家长机器人”，用于回答孩子们的问题。**Claude 的默认回复相当正式和机械**。这会伤了孩子的心。

In [ ]:
# 提示
PROMPT = "圣诞老人会给我带礼物吗？"

# 打印 Claude 的回复
print(get_completion(PROMPT))

你可以花时间描述你想要的语气，但**给 Claude 几个理想回复的例子要容易得多**。

In [ ]:
# 提示
PROMPT = """请通过编写下一行来完成对话，以“A”的身份发言。
Q: 牙仙是真的吗？
A: 当然是真的，亲爱的。把你的牙齿包好，今晚放在枕头下面。明天早上可能会有惊喜等着你哦。
Q: 圣诞老人会给我带礼物吗？"""

# 打印 Claude 的回复
print(get_completion(PROMPT))

在下面的格式化示例中，我们可以一步步地向 Claude 详细说明如何提取姓名和职业，然后按照我们想要的方式精确格式化它们；或者，我们也可以**直接向 Claude 提供一些格式正确的示例，Claude 就能从中推断出来**。请注意`assistant`回合中的`<individuals>`，这能让 Claude 有一个正确的开始。

In [ ]:
# 带有变量占位符的提示模板
PROMPT = """Silvermist Hollow，一个迷人的村庄，住着一群非凡的人物。
其中有利亚姆·帕特尔医生，一位神经外科医生，他在地区医疗中心彻底改变了手术技术。
奥利维亚·陈是一位创新建筑师，她以可持续且令人惊叹的设计改变了村庄的景观。
当地剧院因伊森·科瓦奇的迷人交响乐而增色不少，他是一位受过专业训练的音乐家和作曲家。
伊莎贝拉·托雷斯，一位自学成才的厨师，对当地食材充满热情，她的农场到餐桌餐厅创造了烹饪轰动，成为美食爱好者必去的目的地。
这些非凡的人物，各自拥有独特的才能，为 Silvermist Hollow 充满活力的生活画卷增添了色彩。
<individuals>
1. 利亚姆·帕特尔医生 [神经外科医生]
2. 奥利维亚·陈 [建筑师]
3. 伊森·科瓦奇 [音乐家和作曲家]
4. 伊莎贝拉·托雷斯 [厨师]
</individuals>

在镇中心，主厨奥利弗·汉密尔顿以他的农场到餐桌餐厅“绿盘”改变了烹饪界。奥利弗对采购当地有机食材的奉献精神为这家餐厅赢得了美食评论家和当地人的好评。
就在街对面，你会发现河畔林地图书馆，那里的首席图书管理员伊丽莎白·陈一直勤奋工作，为所有人创造了一个温馨包容的空间。她为扩大图书馆服务范围和为儿童建立阅读项目所做的努力，对镇上的识字率产生了重大影响。
当你漫步穿过迷人的小镇广场时，会被墙上美丽的壁画所吸引。这些杰作是著名艺术家伊莎贝拉·托雷斯的作品，她捕捉河畔林地精髓的才华使小镇焕发生机。
河畔林地的体育成就也值得一提，这要归功于前奥运游泳运动员转型的教练马库斯·詹金斯。马库斯利用他的经验和热情训练镇上的年轻人，带领河畔林地游泳队赢得了数个地区冠军。
<individuals>
1. 奥利弗·汉密尔顿 [主厨]
2. 伊丽莎白·陈 [图书管理员]
3. 伊莎贝拉·托雷斯 [艺术家]
4. 马库斯·詹金斯 [教练]
</individuals>

橡树谷，一个迷人的小镇，住着三位杰出的人物，他们的技能和奉献精神对社区产生了持久的影响。
在镇上熙熙攘攘的农贸市场，你会发现劳拉·西蒙斯，一位热情的有机农民，以她美味和可持续种植的农产品而闻名。她致力于推广健康饮食，激励小镇居民拥抱更环保的生活方式。
在橡树谷的社区中心，凯文·阿尔瓦雷斯，一位技艺精湛的舞蹈教练，为各个年龄段的人们带来了运动的乐趣。他包容性的舞蹈课程培养了居民的团结感和自我表达能力，丰富了当地的艺术氛围。
最后，蕾切尔·奥康纳，一位不知疲倦的志愿者，将她的时间奉献给各种慈善事业。她致力于改善他人生活的承诺，在橡树谷内建立了强大的社区意识。
通过他们独特的才能和坚定不移的奉献精神，劳拉、凯文和蕾切尔已经融入了橡树谷的肌理，帮助创造了一个充满活力和繁荣的小镇。"""

# Claude 回复的预填充
PREFILL = "<individuals>"

# 打印 Claude 的回复
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print("用户回合：")
print(PROMPT)
print("\n 助手回合：")
print(PREFILL)
print("\n------------------------------------- Claude 的回复 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

如果您想在不更改上方任何内容的情况下，尝试本课程的提示，请一直滚动到课程笔记本的底部，访问[**示例演练场**](#example-playground)。


---

## 练习
- [练习 7.1 - 通过示例进行电子邮件格式化](#exercise-71---email-formatting-via-examples)

### 练习 7.1 - 通过示例进行电子邮件格式化
我们将重新做练习 6.2，但这次，我们将编辑`PROMPT`，使用电子邮件的“少样本”示例 + 正确的分类（和格式化），以使 Claude 输出正确答案。我们希望 Claude 输出的**最后一个字母**是类别的字母。

如果您忘记了每封电子邮件对应的正确类别字母，请参考`EMAILS`列表中每封电子邮件旁边的注释。

请记住，这些是电子邮件的类别：
- (A) 售前问题
- (B) 损坏或有缺陷的商品
- (C) 账单问题
- (D) 其他（请解释）

In [ ]:
# 带有变量占位符的提示模板
PROMPT = """请将此电子邮件分类为绿色或蓝色：{email}"""

# Claude 回复的预填充
PREFILL = ""

# 存储为列表的变量内容
EMAILS = [
    "嗨——我的 Mixmaster4000 运行时发出奇怪的噪音。它还有点烟味和塑料味，像电子设备烧焦了。我需要一个替换品。", # (B) 损坏或有缺陷的商品
    "我可以用我的 Mixmaster 4000 搅拌油漆吗，还是它只用于搅拌食物？", # (A) 售前问题 或 (D) 其他（请解释）
    "我取消后已经等了 4 个月，我的月度费用怎么还没停下来！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这儿来的，我不太会用电脑。救命。" # (D) 其他（请解释）
]

# 存储为列表的正确分类列表，以适应每封电子邮件可能存在多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 遍历电子邮件列表
for i,email in enumerate(EMAILS):
    
    # 将电子邮件文本替换到电子邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取 Claude 的回复
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 评估 Claude 的回复
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # 打印 Claude 的回复
    print("--------------------------- 带有变量替换的完整提示 ---------------------------")
    print("用户回合")
    print(formatted_prompt)
    print("\n 助手回合")
    print(PREFILL)
    print("\n------------------------------------- Claude 的回复 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决：", grade, "\n\n\n\n\n\n")

❓ 如果您想要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

还是卡住了？运行下面的单元格以获取示例解决方案。

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已准备好进入下一章。祝您提示愉快！

---

# Chapter 7: Using Examples (Few-Shot Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

**Giving Claude examples of how you want it to behave (or how you want it not to behave) is extremely effective** for:
- Getting the right answer
- Getting the answer in the right format

This sort of prompting is also called "**few shot prompting**". You might also encounter the phrase "zero-shot" or "n-shot" or "one-shot". The number of "shots" refers to how many examples are used within the prompt.

### Examples

Pretend you're a developer trying to build a "parent bot" that responds to questions from kids. **Claude's default response is quite formal and robotic**. This is going to break a child's heart.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Claude's response
print(get_completion(PROMPT))

You could take the time to describe your desired tone, but it's much easier just to **give Claude a few examples of ideal responses**.

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

In the following formatting example, we could walk Claude step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just **provide Claude with some correctly-formatted examples and Claude can extrapolate from there**. Note the `<individuals>` in the `assistant` turn to start Claude off on the right foot.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for Claude's response
PREFILL = "<individuals>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 7.1 - Email Formatting via Examples](#exercise-71---email-formatting-via-examples)

### Exercise 7.1 - Email Formatting via Examples
We're going to redo Exercise 6.2, but this time, we're going to edit the `PROMPT` to use "few-shot" examples of emails + proper classification (and formatting) to get Claude to output the correct answer. We want the *last* letter of Claude's output to be the letter of the category.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

Remember that these are the categories for the emails:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)								

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

Still stuck? Run the cell below for an example solution.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for Claude's response
PREFILL = "<individuals>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))